# Moosic EDA :: Iteration v1



## Importing required libraries

* numpy
* pandas
* ??
* scikit learn

In [1]:
# IMPORT LIBRARIES


try:

    import numpy as np
    import pandas as pd

    # visualisation
    import seaborn as sns
    import matplotlib.pyplot as plt
    from sklearn.model_selection import train_test_split


except ImportError as error:
    print(f"Installation of the required dependencies necessary! {error}")

    %pip install numpy
    %pip install pandas
    %pip install seaborn
    %pip install matplotlib

    print(f"Successful installation of the required dependencies necessary")


    import warnings
    warnings.filterwarnings('ignore')




Import the Datasets

In [2]:
df_artists = pd.read_csv('../.data/artists.csv', low_memory=False)
df_tracks = pd.read_csv('../.data/tracks.csv', low_memory=False)

## Data Overview Artists

| column | additional information |
|--------|------------------------|
| id | id of artist |
| followers | number of followers | 
| genres | genres associated with artist |
| name | name of artist |
| popularity | popularity of artist in range 0 to 100 |

## Data Overview Tracks

| column | additional information |
|--------|------------------------|
| id | id of track |
| name | name of track | 
| popularity | popularity of track in range 0 to 100 |
| duration_ms | duration of songs in ms |
| explicit | whether it contains explicit content or not |
| artists | artists who created the track | 
| id_artists | id of artists who created the track |
| release_date | date of release |
| danceability | how danceable a song is in range 0 to 1 |
| energy | how energized a song is in range 0 to 1 |
| key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1 |
| loudness | The overall loudness of a track in decibels (dB) |
| mode |  Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0 |
| speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks |
| acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic |
| instrumentalness | Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content |
| liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live |
| valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry) |
| tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration | 
| time_signature | An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of 3/4, to 7/4. | 

In [3]:
df_artists.columns

Index(['id', 'followers', 'genres', 'name', 'popularity'], dtype='object')

In [4]:
df_tracks.columns

Index(['id', 'name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')

Get general information from df_artists:

In [5]:
df_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162095 entries, 0 to 1162094
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   id          1162095 non-null  object 
 1   followers   1162084 non-null  float64
 2   genres      1162095 non-null  object 
 3   name        1162092 non-null  object 
 4   popularity  1162095 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 44.3+ MB


In [6]:
df_artists.head()

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0


In [7]:
df_artists.describe()

,followers,popularity
count,1.162084e+06,1.162095e+06
mean,1.022070e+04,8.795961e+00
std,2.543995e+05,1.355777e+01
min,0.000000e+00,0.000000e+00
25%,1.000000e+01,0.000000e+00
50%,5.700000e+01,2.000000e+00
75%,4.170000e+02,1.300000e+01
max,7.890023e+07,1.000000e+02


Get general information from df_tracks:

In [8]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 20 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                586672 non-null  object 
 1   name              586601 non-null  object 
 2   popularity        586672 non-null  int64  
 3   duration_ms       586672 non-null  int64  
 4   explicit          586672 non-null  int64  
 5   artists           586672 non-null  object 
 6   id_artists        586672 non-null  object 
 7   release_date      586672 non-null  object 
 8   danceability      586672 non-null  float64
 9   energy            586672 non-null  float64
 10  key               586672 non-null  int64  
 11  loudness          586672 non-null  float64
 12  mode              586672 non-null  int64  
 13  speechiness       586672 non-null  float64
 14  acousticness      586672 non-null  float64
 15  instrumentalness  586672 non-null  float64
 16  liveness          58

In [9]:
df_tracks.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [10]:
df_tracks.describe()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,586672.000000,5.866720e+05,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000,586672.000000
mean,27.570053,2.300512e+05,0.044086,0.563594,0.542036,5.221603,-10.206067,0.658797,0.104864,0.449863,0.113451,0.213935,0.552292,118.464857,3.873382
std,18.370642,1.265261e+05,0.205286,0.166103,0.251923,3.519423,5.089328,0.474114,0.179893,0.348837,0.266868,0.184326,0.257671,29.764108,0.473162
min,0.000000,3.344000e+03,0.000000,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,1.750930e+05,0.000000,0.453000,0.343000,2.000000,-12.891000,0.000000,0.034000,0.096900,0.000000,0.098300,0.346000,95.600000,4.000000
50%,27.000000,2.148930e+05,0.000000,0.577000,0.549000,5.000000,-9.243000,1.000000,0.044300,0.422000,0.000024,0.139000,0.564000,117.384000,4.000000
75%,41.000000,2.638670e+05,0.000000,0.686000,0.748000,8.000000,-6.482000,1.000000,0.076300,0.785000,0.009550,0.278000,0.769000,136.321000,4.000000
max,100.000000,5.621218e+06,1.000000,0.991000,1.000000,11.000000,5.376000,1.000000,0.971000,0.996000,1.000000,1.000000,1.000000,246.381000,5.000000


As we want all necessary information in a single dataset, we need to combine the genre and followers columns from the artists df with the tracks df.

We can do this by using the artist's id's from both dataframes. But first we need to make sure all entries are in the same format. In the df_artists 'genre' and in df_tracks 'artists' and 'id_artists' entries seem to be in this format ['...']. 

In [11]:
# Remove square brackets and quotes from the entire df_tracks
df_tracks = df_tracks.applymap(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ""))

In [12]:
df_tracks.head().T

,0,1,2,3,4
id,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
name,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
popularity,6,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
explicit,0,0,0,0,0
artists,Uli,Fernando Pessoa,Ignacio Corsini,Ignacio Corsini,Dick Haymes
id_artists,45tIt06XoI0Iio4LBEVpls,14jtPCOoNZwquk5wd9DxrY,5LiOoJbxVSAMkBS2fUm3X2,5LiOoJbxVSAMkBS2fUm3X2,3BiJGZsyX9sJchTqcSA7Su
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


In [13]:
# Remove square brackets and quotes from the entire df_tracks
df_artists = df_artists.applymap(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ""))

In [14]:
df_artists.head().T

,0,1,2,3,4
id,0DheY5irMjBUeLybbCUEZ2,0DlhY15l3wsrnlfGio2bjU,0DmRESX2JknGPQyO15yxg7,0DmhnbHjm1qw6NCYPeZNgJ,0Dn11fWM7vHQ3rinvWEl4E
followers,0.0,5.0,0.0,0.0,2.0
genres,,,,,
name,Armid & Amir Zare Pashai feat. Sara Rouzbehani,ปูนา ภาวิณี,Sadaa,Tragruda,Ioannis Panoutsopoulos
popularity,0,0,0,0,0


Now that all the entries should be in the same 'clean' format, we can merge the 2 sets by the artists id:

In [15]:
# Merge df_artists and df_tracks using 'id' from df_artists and 'id_artists' from df_tracks
combined_df = df_tracks.merge(df_artists, left_on='id_artists', right_on='id', how='left')

In [16]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586672 entries, 0 to 586671
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_x              586672 non-null  object
 1   name_x            586672 non-null  object
 2   popularity_x      586672 non-null  object
 3   duration_ms       586672 non-null  object
 4   explicit          586672 non-null  object
 5   artists           586672 non-null  object
 6   id_artists        586672 non-null  object
 7   release_date      586672 non-null  object
 8   danceability      586672 non-null  object
 9   energy            586672 non-null  object
 10  key               586672 non-null  object
 11  loudness          586672 non-null  object
 12  mode              586672 non-null  object
 13  speechiness       586672 non-null  object
 14  acousticness      586672 non-null  object
 15  instrumentalness  586672 non-null  object
 16  liveness          586672 non-null  obj

In [17]:
combined_df.head().T

,0,1,2,3,4
id_x,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
name_x,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
popularity_x,6,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
explicit,0,0,0,0,0
artists,Uli,Fernando Pessoa,Ignacio Corsini,Ignacio Corsini,Dick Haymes
id_artists,45tIt06XoI0Iio4LBEVpls,14jtPCOoNZwquk5wd9DxrY,5LiOoJbxVSAMkBS2fUm3X2,5LiOoJbxVSAMkBS2fUm3X2,3BiJGZsyX9sJchTqcSA7Su
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


Now let's check for null values and duplicates in the new combined_df

In [18]:
# Check for null values in the dataframe
null_counts = combined_df.isnull().sum()

# Check for duplicate rows in the dataframe
duplicate_counts = combined_df.duplicated().sum()

print("Null value counts:")
print(null_counts)

print("\nNumber of duplicate rows:", duplicate_counts)

Null value counts:
id_x                     0
name_x                   0
popularity_x             0
duration_ms              0
explicit                 0
artists                  0
id_artists               0
release_date             0
danceability             0
energy                   0
key                      0
loudness                 0
mode                     0
speechiness              0
acousticness             0
instrumentalness         0
liveness                 0
valence                  0
tempo                    0
time_signature           0
id_y                116634
followers           116634
genres              116634
name_y              116634
popularity_y        116634
dtype: int64

Number of duplicate rows: 0


There are many null values after our join, lets go more into detail with this:

In [19]:
# Print rows with null values in the columns from df_artists
null_rows = combined_df[combined_df['id_y'].isnull()]
print("Rows with null values in df_artists columns:")
null_rows.head().T

Rows with null values in df_artists columns:


,36,37,84,96,130
id_x,1MD0Obbza9l0t0Zpgcwagy,1O9iZyzufN1fUdVO97mmm5,3h8ioTTWfrC25hrHwQQpLc,4DE7VBCgyTjqV0Plc9Ra0U,6GhbZRE9N2TPf86zI1dIxQ
name_x,And Mimi,How High the Moon,All Or Nothing at All,Hush-A-Bye (Wee Rose of Kilarney),Youll Never Know
popularity_x,0,0,0,0,0
duration_ms,186147,175333,164320,156880,165253
explicit,0,0,0,0,0
artists,"Dick Haymes, Gordon Jenkins, His Orchestra","Dick Haymes, Harry James, His Orchestra","Dick Haymes, Harry James, His Orchestra","Dick Haymes, Victor Young, His Orchestra","Dick Haymes, The Pied Pipers"
id_artists,"3BiJGZsyX9sJchTqcSA7Su, 58wzyK6DupVsypvs3QV2Fo...","3BiJGZsyX9sJchTqcSA7Su, 5MpELOfAiq7aIBTij30phD...","3BiJGZsyX9sJchTqcSA7Su, 5MpELOfAiq7aIBTij30phD...","3BiJGZsyX9sJchTqcSA7Su, 3HqN7Sq7rmpOEI9UV5ERuz...","3BiJGZsyX9sJchTqcSA7Su, 5o8E07TcYqIefZpDejymAd"
release_date,1922,1922,1922,1922,1922
danceability,0.284,0.328,0.237,0.279,0.392
energy,0.223,0.307,0.555,0.128,0.128


We can throw them out, because there are no matching id's ...

In [20]:
combined_df_cleaned = combined_df.dropna()

In [21]:
combined_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 470038 entries, 0 to 586671
Data columns (total 25 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_x              470038 non-null  object
 1   name_x            470038 non-null  object
 2   popularity_x      470038 non-null  object
 3   duration_ms       470038 non-null  object
 4   explicit          470038 non-null  object
 5   artists           470038 non-null  object
 6   id_artists        470038 non-null  object
 7   release_date      470038 non-null  object
 8   danceability      470038 non-null  object
 9   energy            470038 non-null  object
 10  key               470038 non-null  object
 11  loudness          470038 non-null  object
 12  mode              470038 non-null  object
 13  speechiness       470038 non-null  object
 14  acousticness      470038 non-null  object
 15  instrumentalness  470038 non-null  object
 16  liveness          470038 non-null  object
 

In [22]:
combined_df_cleaned.head().T

,0,1,2,3,4
id_x,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
name_x,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
popularity_x,6,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
explicit,0,0,0,0,0
artists,Uli,Fernando Pessoa,Ignacio Corsini,Ignacio Corsini,Dick Haymes
id_artists,45tIt06XoI0Iio4LBEVpls,14jtPCOoNZwquk5wd9DxrY,5LiOoJbxVSAMkBS2fUm3X2,5LiOoJbxVSAMkBS2fUm3X2,3BiJGZsyX9sJchTqcSA7Su
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


We need to rename some collumns:

In [23]:
# Rename the columns
combined_df_cleaned.rename(columns={
    'id_x': 'track_id',
    'id_artists': 'artists_id',
    'name_x': 'track_name',
    'artists': 'artist_name',
    'popularity_x': 'artist_popularity',
    'popularity_y': 'track_popularity'
}, inplace=True)

/var/folders/3m/ptnjs1xn4hjdtk5jszdmj7c00000gn/T/ipykernel_3400/3978385934.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_cleaned.rename(columns={


In [24]:
combined_df_cleaned.head().T

,0,1,2,3,4
track_id,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
track_name,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
artist_popularity,6,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
explicit,0,0,0,0,0
artist_name,Uli,Fernando Pessoa,Ignacio Corsini,Ignacio Corsini,Dick Haymes
artists_id,45tIt06XoI0Iio4LBEVpls,14jtPCOoNZwquk5wd9DxrY,5LiOoJbxVSAMkBS2fUm3X2,5LiOoJbxVSAMkBS2fUm3X2,3BiJGZsyX9sJchTqcSA7Su
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


drop columns we don't need anymore:

In [25]:
df_cleaned_1 = combined_df_cleaned.drop(['name_y', 'id_y'], axis=1)

In [26]:
df_cleaned_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 470038 entries, 0 to 586671
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   track_id           470038 non-null  object
 1   track_name         470038 non-null  object
 2   artist_popularity  470038 non-null  object
 3   duration_ms        470038 non-null  object
 4   explicit           470038 non-null  object
 5   artist_name        470038 non-null  object
 6   artists_id         470038 non-null  object
 7   release_date       470038 non-null  object
 8   danceability       470038 non-null  object
 9   energy             470038 non-null  object
 10  key                470038 non-null  object
 11  loudness           470038 non-null  object
 12  mode               470038 non-null  object
 13  speechiness        470038 non-null  object
 14  acousticness       470038 non-null  object
 15  instrumentalness   470038 non-null  object
 16  liveness           470038

In [27]:
df_cleaned_1.columns

Index(['track_id', 'track_name', 'artist_popularity', 'duration_ms',
       'explicit', 'artist_name', 'artists_id', 'release_date', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'followers', 'genres', 'track_popularity'],
      dtype='object')

Rearrange the order of columns

In [28]:
# Define the desired column order
desired_column_order = ['artists_id', 'track_id', 'artist_name', 'track_name', 'genres', 'release_date', 'explicit', 'duration_ms',
                        'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
                        'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'followers',
                        'artist_popularity', 'track_popularity']

# Rearrange the columns
df_reordered = df_cleaned_1[desired_column_order]

In [29]:
df_reordered.head().T

,0,1,2,3,4
artists_id,45tIt06XoI0Iio4LBEVpls,14jtPCOoNZwquk5wd9DxrY,5LiOoJbxVSAMkBS2fUm3X2,5LiOoJbxVSAMkBS2fUm3X2,3BiJGZsyX9sJchTqcSA7Su
track_id,35iwgR4jXetI318WEWsa1Q,021ht4sdgPcrDgSk7JTbKY,07A5yehtSnoedViJAZkNnc,08FmqUhxtyLTn6pAh6bk45,08y9GfoqCWfOGsKdwojr5e
artist_name,Uli,Fernando Pessoa,Ignacio Corsini,Ignacio Corsini,Dick Haymes
track_name,Carve,Capítulo 2.16 - Banquero Anarquista,Vivo para Quererte - Remasterizado,El Prisionero - Remasterizado,Lady of the Evening
genres,,,"tango, vintage tango","tango, vintage tango","adult standards, big band, easy listening, lou..."
release_date,1922-02-22,1922-06-01,1922-03-21,1922-03-21,1922
explicit,0,0,0,0,0
duration_ms,126903,98200,181640,176907,163080
danceability,0.645,0.695,0.434,0.321,0.402
energy,0.445,0.263,0.177,0.0946,0.158


Now we want to make sure, the most popular artists are still presented in our dataset after the cleaning process:
Therefor we can compare our Dataset with the artist from the "Most Streamed Artist Table" 

In [30]:
# Read the HTML table with the Spotify most streamed artists of all time
url = 'https://kworb.net/spotify/artists.html'
df_ms_artists = pd.read_html(url)[0]

In [31]:
df_ms_artists.head().T

,0,1,2,3,4
Artist,Drake,Bad Bunny,Taylor Swift,The Weeknd,Ed Sheeran
Streams,84273.9,66822.8,56376.6,52956.0,47630.5
Daily,48.712,44.886,97.767,45.427,17.889
As lead,56726.9,40568.4,54104.5,42124.2,42533.5
Solo,32419.9,22830.1,49063.1,30827.5,33723.2
As feature,27547.1,26254.4,2272.1,10831.8,5097.1


We have to compare: How many Artist are in this Table

In [34]:
# Get the unique artists from df_ms_artists
ms_unique_artists = df_ms_artists['Artist'].unique()

# Count how many of these unique artists are in combined_df_final
matching_artist_count = df_reordered['artist_name'].isin(ms_unique_artists).sum()

print("Number of artists from df_ms_artists in combined_df_final:", matching_artist_count)

Number of artists from df_ms_artists in combined_df_final: 96528


Seems that there are enough similarities to go on with our dataset.

In [37]:
# df_reordered
# Find the most presented artists
most_presented_artists = df_reordered['artist_name'].value_counts()

# Find the most presented genres
most_presented_genres = df_reordered['genres'].value_counts()

print("Most presented artists:")
print(most_presented_artists)

print("\nMost presented genres:")
print(most_presented_genres)

Most presented artists:
artist_name
Die drei ???         3856
TKKG Retro-Archiv    2006
Benjamin Blümchen    1503
Bibi Blocksberg      1472
Lata Mangeshkar      1373
                     ... 
Mike Sarkissian         1
Peder Alhaug            1
Herta Marshall          1
BanjoNoah               1
ROLE MODEL              1
Name: count, Length: 55111, dtype: int64

Most presented genres:
genres
                                                                       37810
hoerspiel                                                               8027
kleine hoerspiel                                                        2081
classic italian pop, italian adult pop                                  1781
classic israeli pop, israeli rock                                       1725
                                                                       ...  
baglama, turkish classical, turkish folk                                   1
banjo, traditional country, traditional folk                       

There are a lot of 'Hörspiele' in our Dataset, lets try to remove them and see how many data is lost due to this.

In [39]:
df_reordered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 470038 entries, 0 to 586671
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   artists_id         470038 non-null  object
 1   track_id           470038 non-null  object
 2   artist_name        470038 non-null  object
 3   track_name         470038 non-null  object
 4   genres             470038 non-null  object
 5   release_date       470038 non-null  object
 6   explicit           470038 non-null  object
 7   duration_ms        470038 non-null  object
 8   danceability       470038 non-null  object
 9   energy             470038 non-null  object
 10  key                470038 non-null  object
 11  loudness           470038 non-null  object
 12  mode               470038 non-null  object
 13  speechiness        470038 non-null  object
 14  acousticness       470038 non-null  object
 15  instrumentalness   470038 non-null  object
 16  liveness           470038

In [40]:
# Create a boolean mask for rows with 'hoerspiel' in the 'genres' column
mask = df_reordered['genres'].str.contains('hoerspiel', case=False)

# Filter the dataframe to exclude rows with 'hoerspiel' in the 'genres' column
df_filtered = df_reordered[~mask]

In [41]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 457150 entries, 0 to 586671
Data columns (total 23 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   artists_id         457150 non-null  object
 1   track_id           457150 non-null  object
 2   artist_name        457150 non-null  object
 3   track_name         457150 non-null  object
 4   genres             457150 non-null  object
 5   release_date       457150 non-null  object
 6   explicit           457150 non-null  object
 7   duration_ms        457150 non-null  object
 8   danceability       457150 non-null  object
 9   energy             457150 non-null  object
 10  key                457150 non-null  object
 11  loudness           457150 non-null  object
 12  mode               457150 non-null  object
 13  speechiness        457150 non-null  object
 14  acousticness       457150 non-null  object
 15  instrumentalness   457150 non-null  object
 16  liveness           457150